In [20]:
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F

In [21]:
def encode_onehot(labels):
    # The classes must be sorted before encoding to enable static class encoding.
    # In other words, make sure the first class always maps to index 0.
    classes = sorted(list(set(labels)))
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

In [22]:
def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)


def normalize_features(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [23]:
def load_data(path="./cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset), dtype=np.dtype(str))

    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset), dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize_features(features)
    adj = normalize_adj(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    adj = torch.FloatTensor(np.array(adj.todense()))
    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

In [24]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [25]:
print(adj.shape, features.shape, labels.shape, idx_train.shape, idx_val.shape, idx_test.shape)

torch.Size([2708, 2708]) torch.Size([2708, 1433]) torch.Size([2708]) torch.Size([140]) torch.Size([300]) torch.Size([1000])


### GAT model

In [26]:
class GATLayer(nn.Module):

    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GATLayer, self).__init__()
        self.in_features = in_features  # 节点向量的特征维度
        self.out_features = out_features  # 经过GAT之后的特征维度
        self.dropout = dropout  # dropout参数
        self.alpha = alpha  # leakyReLU的参数

        # 定义可训练参数，即论文中的W和a
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)  # xavier初始化
        self.a = nn.Parameter(torch.zeros(size=(2 * out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)  # xavier初始化

        # 定义leakyReLU激活函数
        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, input_h, adj):
        """
        input_h:  [N, in_features]
        adj: 图的邻接矩阵 维度[N, N] 非零即一，可以参考5分钟-通俗易懂-图神经网络计算
        """
        # self.W [in_features,out_features]
        # input_h × self.W  ->  [N, out_features]

        h = torch.mm(input_h, self.W)  # [N, out_features]

        N = h.size()[0]  # N 图的节点数
        input_concat = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1). \
            view(N, -1, 2 * self.out_features)

        # [N, N, 2*out_features]
        e = self.leakyrelu(torch.matmul(input_concat, self.a).squeeze(2))
        # [N, N, 1] => [N, N] 图注意力的相关系数（未归一化）

        zero_vec = -1e12 * torch.ones_like(e)  # 将没有连接的边置为负无穷
        attention = torch.where(adj > 0, e, zero_vec)  # [N, N]
        # 表示如果邻接矩阵元素大于0时，则两个节点有连接，该位置的注意力系数保留，
        # 否则需要mask并置为非常小的值，原因是softmax的时候这个最小值会不考虑。
        attention = F.softmax(attention, dim=1)  # softmax形状保持不变 [N, N]，得到归一化的注意力权重！
        attention = F.dropout(attention, self.dropout, training=self.training)  # dropout，防止过拟合
        output_h = torch.matmul(attention, h)  # [N, N].[N, out_features] => [N, out_features]
        # 得到由周围节点通过注意力权重进行更新的表示
        return output_h

In [27]:
class MyGAT(nn.Module):
    def __init__(self, input_feature_size, output_size, nclass, dropout, alpha, nheads):
        """Dense version of GAT."""
        super(MyGAT, self).__init__()
        self.dropout = dropout

        self.attentions = [GATLayer(input_feature_size, output_size, dropout=dropout, alpha=alpha, concat=True)
                           for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = GATLayer(output_size * nheads, nclass, dropout=dropout, alpha=alpha, concat=False)

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.out_att(x, adj))
        return F.log_softmax(x, dim=1)

In [28]:
import argparse
# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False, help='Disables CUDA training.')
# parser.add_argument('--fastmode', action='store_true', default=False, help='Validate during training pass.')
parser.add_argument('--model', type=str, default="gat", help='which gnn model use')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=10, help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.01, help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=1e-4, help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16, help='Number of hidden units.')
parser.add_argument('--nb_heads', type=int, default=8, help='Number of head attentions.')
parser.add_argument('--dropout', type=float, default=0.6, help='Dropout rate (1 - keep probability).')
parser.add_argument('--alpha', type=float, default=0.2, help='Alpha for the leaky_relu.')
parser.add_argument('--patience', type=int, default=100, help='Patience')
# parser.add_argument('--config', type=str, default='./experiments.conf')   # 获取的一些配置文件
#args = parser.parse_args()                                               # pychram 中使用
args = parser.parse_known_args()[0]                                       # jupyter 中使用
args

Namespace(no_cuda=False, model='gat', seed=42, epochs=10, lr=0.01, weight_decay=0.0001, hidden=16, nb_heads=8, dropout=0.6, alpha=0.2, patience=100)

In [29]:
import random
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)

In [30]:
print(" === 使用 自己构建的 gat ==== ")
model = MyGAT(input_feature_size=1433,
            output_size=12,
            nclass=7,
            dropout=0.6,
            nheads=8,
            alpha=0.2)

 === 使用 自己构建的 gat ==== 


In [31]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr,
                       weight_decay=args.weight_decay)

In [32]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [33]:
def train(epoch):
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    # if not args.fastmode:
    #     # Evaluate validation set performance separately,
    #     # deactivates dropout during validation run.
    #     model.eval()
    #     output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.data.item()),
          'acc_train: {:.4f}'.format(acc_train.data.item()),
          'loss_val: {:.4f}'.format(loss_val.data.item()),
          'acc_val: {:.4f}'.format(acc_val.data.item()))

    return loss_val.data.item()

In [34]:
def compute_test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data.item()),
          "accuracy= {:.4f}".format(acc_test.data.item()))

In [35]:
loss_values = []
bad_counter = 0
best = args.epochs + 1
best_epoch = 0
for epoch in range(args.epochs):
    loss_values.append(train(epoch))

    # torch.save(model.state_dict(), '{}.pkl'.format(epoch))
    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == args.patience:
        break


print("Optimization Finished!")



Epoch: 0001 loss_train: 1.9508 acc_train: 0.1643 loss_val: 1.9498 acc_val: 0.1367
Epoch: 0002 loss_train: 1.9219 acc_train: 0.2929 loss_val: 1.9230 acc_val: 0.3133
Epoch: 0003 loss_train: 1.8939 acc_train: 0.4929 loss_val: 1.8974 acc_val: 0.4500
Epoch: 0004 loss_train: 1.8593 acc_train: 0.5429 loss_val: 1.8820 acc_val: 0.4533
Epoch: 0005 loss_train: 1.8204 acc_train: 0.6071 loss_val: 1.8560 acc_val: 0.5200
Epoch: 0006 loss_train: 1.7883 acc_train: 0.5143 loss_val: 1.8196 acc_val: 0.5433
Epoch: 0007 loss_train: 1.7395 acc_train: 0.6357 loss_val: 1.7935 acc_val: 0.5433
Epoch: 0008 loss_train: 1.7324 acc_train: 0.5786 loss_val: 1.7785 acc_val: 0.5067
Epoch: 0009 loss_train: 1.6587 acc_train: 0.5929 loss_val: 1.7685 acc_val: 0.5167
Epoch: 0010 loss_train: 1.6688 acc_train: 0.5857 loss_val: 1.7071 acc_val: 0.4833
Optimization Finished!


In [36]:
# Testing
compute_test()

Test set results: loss= 1.7122 accuracy= 0.4760
